In [3]:
import numpy as np
from matplotlib import cm
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs
from rdkit.Chem.Draw import SimilarityMaps
from sklearn import svm
from sklearn.model_selection import train_test_split
import os
from sklearn.externals import joblib
# dense to sparse
from numpy import array
from scipy.sparse import csr_matrix

import keras
import tensorflow as tf
from sklearn import preprocessing
from keras import regularizers
abscaler= preprocessing.MaxAbsScaler()
from os import environ


import csv

f = open('csvfile.csv')
csv_f = csv.reader(f)
i=0
args=[]
for row in csv_f:
    args+=row
    
nodes=np.int64(args[0])  
dropout=float(args[1])
regularizers=float(args[2])
lr=float(args[3])
momentum=float(args[4])
decay=float(args[5])
mynesterov=args[6]
theloss=args[7]
epochs=np.int64(args[8])
batchsize=np.int64(args[9])
filename=args[10]
modelname=args[11]

def process(filename,modelname, dropout, nodes, regularizers, epochs, batchsize, lr, momentum, decay, mynesterov):
    

    

    dataset = open(filename, "r" )
    dataset = [ line.rstrip().split(",") for line in dataset ][0:]
    mols = [ Chem.MolFromSmiles( line[0] ) for line in dataset ]


    u=0
    indexy=[]
    for y in mols:
        if y is not None:
            indexy.append(u)
            u+=1
        
    goodmols=[mols[k] for k in indexy]
    
   

    Y=[line[1] for line in dataset]
    goody=[Y[k] for k in indexy]
    trainmols, testmols, trainy, testy = train_test_split(goodmols, goody, test_size = 0.1, random_state=90  )
    #trainmols, testmols, trainy, testy = train_test_split(goodmols, Y, test_size = 0.1, random_state=90  )
    trainfps = [AllChem.GetMorganFingerprintAsBitVect(m, 2) for m in goodmols if m is not None ]
    testfps = [AllChem.GetMorganFingerprintAsBitVect(m, 2) for m in testmols if m is not None]
    
    
    
    
   
    u=0
    indexy=[]
    for y in trainfps:
        if y is not None:
            indexy.append(u)
            u+=1
    
    newy=array([int(goody[k]) for k in indexy])
    print(len(newy))
    
    np_fptrain=[]
    for fp in trainfps:
        arr=np.zeros((1,))
        DataStructs.ConvertToNumpyArray(fp, arr)
        np_fptrain.append(arr)

    np_fptest=[]
    for fp in testfps:
        arr=np.zeros((1,))
        DataStructs.ConvertToNumpyArray(fp, arr)
        np_fptest.append(arr)  
    
    
    a=csr_matrix(np_fptrain, dtype=np.int8).toarray()
    b=csr_matrix(np_fptest, dtype=np.int8).toarray()
    a=abscaler.fit_transform(a)
    b=abscaler.fit_transform(b)
    
    print(len(a))
    
    import matplotlib.pyplot as plt
    from sklearn.metrics import mean_squared_error, r2_score

    from sklearn import datasets

   

# In[ ]:


    from sklearn.preprocessing import StandardScaler
    from sklearn.model_selection import cross_validate
    from keras.models import Sequential
    from keras import layers
    from keras.optimizers import SGD




    from keras.models import Sequential
 
    from keras.callbacks import ReduceLROnPlateau
    from keras import regularizers as WeightRegularizer
    from keras.optimizers import SGD
    #SKlearn for metrics and datasplits
    from sklearn.model_selection import cross_validate
    from sklearn.metrics import roc_auc_score, roc_curve
    #Matplotlib for plotting
    from matplotlib import pyplot as plt
    




    model = Sequential()
    model.add(layers.Dense(nodes, init='uniform', activation='relu', kernel_regularizer=keras.regularizers.l2(l=regularizers)))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(dropout))
    model.add(layers.Dense(nodes, activation='relu', kernel_regularizer=keras.regularizers.l2(l=regularizers)))
    
    model.add(layers.Dense(1,  activation='sigmoid'))
    
    
    
    sgd=SGD(lr, momentum, decay, nesterov=mynesterov)
    model.compile(loss=theloss, optimizer=sgd, metrics=['accuracy'])
    model.fit(a, newy, nb_epoch=epochs, batch_size=batchsize)
    
    themodel=modelname + '.h5'
    model.save(themodel)
   
    from keras.models import load_model
    model = load_model('classifier.h5')
    model.compile(optimizer = 'sgd', loss = 'binary_crossentropy', metrics=['accuracy'])
    from sklearn.metrics import (matthews_corrcoef, 
                             confusion_matrix, 
                             f1_score, 
                             roc_auc_score,
                             accuracy_score,
                             roc_auc_score)
    predy=model.predict(b)
    predy[predy > 0.5] = 1
    predy[predy <= 0.5] = 0
    testy=np.delete(testy, -1)
    testb=[int(h) for h in testy]
                
    tn, fp, fn, tp = confusion_matrix(testb, predy).ravel()
    sensitivity=tp/(tp + fn)
    specificity= tn/(tn + fp)
    accuracy=(tp+tn)/(tn+fp + fn +tp)
                
    sample = open('samplefile.txt', 'w') 
                
                
    print("accuracy", accuracy, file=sample)
    print("sensitivity", sensitivity, file=sample)
    print("specificity", specificity, file=sample)
                
    sample.close() 
##########################################
    

process(filename,modelname, dropout, nodes, regularizers, epochs, batchsize, lr, momentum, decay, mynesterov)   
    
    
    
 

    
    

IndexError: list assignment index out of range

In [2]:
#from keras.models import load_model
#model = load_model('classifier.h5')
#model.compile(optimizer = 'sgd', loss = 'binary_crossentropy', metrics=['accuracy'])

In [9]:
#tn=5388
#fp=8
#fn=27
#tp=88
#sensitivity=tp/(tp + fn)
#specificity= tn/(tn + fp)
#accuracy=(tp+tn)/(tn+fp + fn +tp)
#print("accuracy", accuracy)
#print("sensitivity", sensitivity)
#print("specificity", specificity)

accuracy 0.9936490655053529
sensitivity 0.7652173913043478
specificity 0.9985174203113417


In [5]:
#import pandas as pd
#import numpy 
#newdf=pd.read_csv('gsk3new.txt', sep='\t', names=['smiles', 'value'])
#def predictor(myinput):
#    smile=myinput
        
#    themol=Chem.MolFromSmiles(smile)
#    estimator = load_model('classifier.h5')
    
#    fp = AllChem.GetMorganFingerprintAsBitVect(themol,2,nBits=2048)
#    res = numpy.zeros(len(fp),numpy.int32)
#    DataStructs.ConvertToNumpyArray(fp,res)
#    probas = list(estimator.predict(res.reshape(1,-1))[0])
#    print(probas)



In [ ]:
#from rdkit import Chem
#import numpy as np
#from rdkit.Chem import AllChem, DataStructs
#validate=newdf['smiles']
#testvalidate=newdf['value']
#mypreds=[]
#for y in validate:
#    a=predictor(y)
#    mypreds.append(a)
#print(mypreds)


In [18]:
#from rdkit import Chem
#import numpy as np
#from rdkit.Chem import AllChem, DataStructs
#newdf=pd.read_csv('testset.txt', names=['smiles'])
#validate=newdf['smiles']
#for h in validate:
#    predictor(h)
    
    


[0.00056889275]
[0.011415197]
[0.00022030345]
[0.31587937]
[0.00012703771]
[0.16461712]
[0.038860872]
[0.00059711566]
[0.43126932]
[0.012150672]
